Playing Ms. Pac-Man with reinforcement learning
> Josh Sowers

> CSC 594 Section 801

> Final Project


In the following notebook, I train an agent to play Ms. Pac-Man using OpenAI's gym and a convolution DDQN. 

# Classes and Imports

## Imports

In [1]:
# First we are going to mount the drive to save the model quickly
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
# Import os and change the directory to the correct folder
import os
os.chdir('/content/gdrive/My Drive/CSC594/')

In [3]:
import random
import gym
import numpy as np
from collections import deque
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPool2D   # Note I do not use MaxPooling
from keras.optimizers import Adam
from keras.utils.vis_utils import plot_model
from google.colab import files

Using TensorFlow backend.


## Classes

### Agent

In [0]:
class Agent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=100000)  # This is currently using 100000 frames as a memory
        
        # Hyperparams
        self.gamma = 0.95           # Discount rate, using slightly lower one to speed short term performance (wont have long term)
        self.epsilon = 1.0          # Exploration rate
        self.epsilon_min = 0.1      # Minimal exploration rate (this is for epsilon-greedy strategy
        self.epsilon_decay = 0.995  # Decay rate for epsilon
        self.update_rate = 10000    # Number of steps until updating the target network
        
        # Initialize double DQN and initializes both with same weights
        self.model = self._build_model()
        self.target_model = self._build_model()
        self.target_model.set_weights(self.model.get_weights())
        self.model.summary() 

    # Build the model
    def _build_model(self):
        model = Sequential()
        
        # Convolution layers first, don't want to lose any information by doing pooling 
        model.add(Conv2D(32, (6, 6), strides=4, padding='same', activation='relu', input_shape=self.state_size))      
        model.add(Conv2D(64, (4, 4), strides=2, padding='same', activation='relu')) 
        model.add(Conv2D(64, (3, 3), strides=1, padding='same', activation='relu'))
        model.add(Flatten())
        # FC layers to learn
        model.add(Dense(128, activation='relu'))
        model.add(Dense(128, activation='relu'))
        model.add(Dense(64, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        
        model.compile(loss='mse', optimizer=Adam(lr=0.001))
        return model

    # Add to memory
    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    # Do the action, epsilon-greedy
    def act(self, state):
        # Random
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        
        # Policy
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])  

    # Trains the model using randomly selected experiences in the replay memory
    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        
        for state, action, reward, next_state, done in minibatch:
            
            if not done:
                # Get the next state
                max_action = np.argmax(self.model.predict(next_state)[0])
                target = (reward + self.gamma * self.target_model.predict(next_state)[0][max_action])
            else:
                target = reward

            # Predict Q-values
            target_df = self.model.predict(state)
            
            # Update target with action using index
            target_df[0][action] = target
            
            # Fit the model - MiniBatch Size will control training
            self.model.fit(state, target_df, epochs=1, verbose=0)
            
        # Epsilon Decay    
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    #Updates target model
    def update_target_model(self):
        self.target_model.set_weights(self.model.get_weights())
            
    #Loads the model
    def load_model(self, name):
        self.model.load_weights(name)

    # Saves the model
    def save_model(self, name):
        self.model.save_weights(name)

### Preprocessing

In [0]:
class Preprocessing:
  # Please note that this preprocessing was adapted from github.com/ageron/tiny-dqn
  # It was extremely helpful in getting the network to work

    def process_frame(self, frame):
        mspacman_color = np.array([210, 164, 74]).mean() # This gets Ms. Pac-Man's colour
        img = frame[1:176:2, ::2]    # Crop and downsize - This gets rid of the bottom of the screen
        img = img.mean(axis=2)       # Convert to greyscale
        img[img==mspacman_color] = 0 # Improve contrast by making pacman white
        img = (img - 128) / 128 - 1  # Normalize from -1 to 1.
        
        return np.expand_dims(img.reshape(88, 80, 1), axis=0)

    def blend_images(self, images, blend):
      # Create an empty image first
      avg_image = np.expand_dims(np.zeros((88, 80, 1), np.float64), axis=0)

      for image in images:
          avg_image += image                # Add images
          
      if len(images) < blend:
          return avg_image / len(images)    # Average of images if we don't have enough yet, so first N images where N < blend
      else:
          return avg_image / blend          # Otherwise blend

# Envrionment and Playing the Game

In [11]:
# Here we make the environment and initialize the agent and Preprocessing class
env = gym.make('MsPacman-v0')
state_size = (88, 80, 1)
action_size = env.action_space.n
agent = Agent(state_size, action_size)
#agent.load_model('agent')

preprocess = Preprocessing()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 22, 20, 32)        1184      
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 11, 10, 64)        32832     
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 11, 10, 64)        36928     
_________________________________________________________________
flatten_3 (Flatten)          (None, 7040)              0         
_________________________________________________________________
dense_9 (Dense)              (None, 128)               901248    
_________________________________________________________________
dense_10 (Dense)             (None, 128)               16512     
_________________________________________________________________
dense_11 (Dense)             (None, 64)               

In [7]:
'''
# This cell builds and saves the model architecture to a file
model = agent._build_model()
plot_model(model, to_file='model.png')
files.download('model.png')
'''

"\n# This cell builds and saves the model architecture to a file\nmodel = agent._build_model()\nplot_model(model, to_file='model.png')\nfiles.download('model.png')\n"

In [0]:
# Global Variables
episodes = 100  # Number of episodes
batch_size = 32  # Batch Size for Network Updating
skip_start = 90  # MsPacman-v0 waits for 90 actions before the episode begins. This is because the first 90 actions are the loading things for the game
total_time = 0   # Counter for total number of steps taken
avg_rewards = 0  # Used to compute avg reward over time
blend = 4        # Number of images to blend
done = False     # Initialize done to False

In [13]:
for i in range(episodes):

    total_reward = 0
    game_score = 0
    state = preprocess.process_frame(env.reset())
    images = deque(maxlen=blend)   # Get the images to blend
    images.append(state)
    
    for skip in range(skip_start): # Skip the start of each game
        env.step(0)
    
    for time in range(20000):      # Only letting it go 20,000 frames. The usual playthrough is 10 million from literature.
        #env.render()              # Have to take out render for Colab
        total_time += 1
        
        # Update target network every update_rate timesteps 
        if total_time % agent.update_rate == 0:
            agent.update_target_model()
        
        # Return the avg of the last blend frames
        state = preprocess.blend_images(images, blend)
        
        # Have the agent take a move
        action = agent.act(state)
        next_state, reward, done, _ = env.step(action)
        
        # Add to reward
        game_score += reward
        total_reward += reward
        
        # Return the average of the last 4 frames
        next_state = preprocess.process_frame(next_state)
        images.append(next_state)
        next_state = preprocess.blend_images(images, blend)
        
        # Store in memory
        agent.remember(state, action, reward, next_state, done)
        
        state = next_state
        
        # Evaluate done state
        if done:
            avg_rewards += game_score
            
            print("episode: {}/{}, game score: {}, reward: {}, avg reward: {}, time: {}, total time: {}"
                  .format(i+1, episodes, game_score, total_reward, avg_rewards/(i+1), time, total_time))
            if i % 10==0:
                agent.save_model('agent')
            break
            
        if len(agent.memory) > batch_size:
            agent.replay(batch_size)

episode: 1/100, game score: 200.0, reward: 200.0, avg reward: 200.0, time: 585, total time: 586
episode: 2/100, game score: 510.0, reward: 510.0, avg reward: 355.0, time: 742, total time: 1329
episode: 3/100, game score: 460.0, reward: 460.0, avg reward: 390.0, time: 728, total time: 2058
episode: 4/100, game score: 240.0, reward: 240.0, avg reward: 352.5, time: 736, total time: 2795
episode: 5/100, game score: 590.0, reward: 590.0, avg reward: 400.0, time: 783, total time: 3579
episode: 6/100, game score: 110.0, reward: 110.0, avg reward: 351.6666666666667, time: 431, total time: 4011
episode: 7/100, game score: 980.0, reward: 980.0, avg reward: 441.42857142857144, time: 907, total time: 4919
episode: 8/100, game score: 640.0, reward: 640.0, avg reward: 466.25, time: 830, total time: 5750
episode: 9/100, game score: 480.0, reward: 480.0, avg reward: 467.77777777777777, time: 642, total time: 6393
episode: 10/100, game score: 440.0, reward: 440.0, avg reward: 465.0, time: 546, total ti

KeyboardInterrupt: ignored